In [185]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [186]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11657633032872606217
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2222702196167190185
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17591022615627754009
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18394245058491585002
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [187]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class BiGRUSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.5):
        super(BiGRUSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(2, self.batch_size, self.hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(2, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        gru_out, self.hidden = self.gru(x, self.hidden)
        y = self.hidden2label(torch.cat((gru_out[-1,:,:150], gru_out[0,:,150:]), 1))
        return y, gru_out, x
        #log_probs = F.log_softmax(y)
        #return log_probs

In [188]:
class GRUSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.7):
        super(GRUSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        gru_out, self.hidden = self.gru(x, self.hidden)
        y = self.hidden2label(gru_out[-1])
        return y, gru_out, x

In [189]:
class LSTMSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.5):
        super(LSTMSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        #self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        #lstm_out = self.dropout(lstm_out)
        y = self.hidden2label(lstm_out[-1])
        return y, lstm_out, x

In [190]:
!pwd


/content


In [191]:
# import torch
# import torch.nn as nn
from torchtext import data
import numpy as np
# from torch.autograd import Variable
from nltk.tokenize import word_tokenize
# import torch.nn.functional as F



def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/REATData/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))


def bigru_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))

    # attribution for the prediction
    weights = best_model.gru.state_dict()

    _, W_iz, _ = np.split(weights['weight_ih_l0'].cpu().numpy(), 3, 0)
    _, W_hz, _ = np.split(weights['weight_hh_l0'].cpu().numpy(), 3, 0)
    _, b_z, _ = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 3)

    _, W_iz_r, _ = np.split(weights['weight_ih_l0_reverse'].cpu().numpy(), 3, 0)
    _, W_hz_r, _ = np.split(weights['weight_hh_l0_reverse'].cpu().numpy(), 3, 0)
    _, b_z_r, _ = np.split(weights['bias_ih_l0_reverse'].cpu().numpy() + weights['bias_hh_l0_reverse'].cpu().numpy(), 3)



    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_iz, x[i,0,:]) + np.matmul(W_hz, hn[i-1,0,:150]) + b_z)
        z_dict.append(z)
    alpha_dict = z_dict

    z_dict_reverse = []
    z_dict_reverse.append(np.ones(150))
    for i in range(length-1):
        i = length - 2 - i
        z = sigmoid(np.matmul(W_iz_r, x[i,0,:]) + np.matmul(W_hz_r, hn[i+1,0,150:]) + b_z_r)
        z_dict_reverse.append(z)
    z_dict_reverse = z_dict_reverse[::-1]
    alpha_dict_reverse = z_dict_reverse

    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()

    
    target_class = pred_label
    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:150]
        else:
            updating = hn[i,0,:150] - alpha_dict[i] * hn[i-1,0,:150]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]

        if i == len(alpha_dict)-1:
            updating_reverse = hn[i,0,150:]
        else:
            updating_reverse = hn[i,0,150:] - alpha_dict_reverse[i] * hn[i+1,0,150:]
        forgetting_reverse = alpha_dict_reverse[-1]
        for j in range(i):
            forgetting_reverse = forgetting_reverse*alpha_dict_reverse[j]

        score = np.matmul( W[target_class], np.concatenate((updating * forgetting,updating_reverse*forgetting_reverse))) #+ b[target_class]
        score_dict.append(score[0])

    return word_tokenize_list, score_dict

In [192]:

def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/REATData/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))



def gru_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()  
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))


    # attribution for the prediction
    weights = best_model.gru.state_dict()
    _, W_iz, _ = np.split(weights['weight_ih_l0'].cpu().numpy(), 3, 0)
    _, W_hz, _ = np.split(weights['weight_hh_l0'].cpu().numpy(), 3, 0)
    _, b_z, _ = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 3)

    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_iz, x[i,0,:]) + np.matmul(W_hz, hn[i-1,0,:]) + b_z)
        z_dict.append(z)
    alpha_dict = z_dict
    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()

    target_class = pred_label
    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:]
        else:
            updating = hn[i,0,:] - alpha_dict[i] * hn[i-1,0,:]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]
        score = np.matmul( W[target_class], updating * forgetting) #+ b[target_class]
        score_dict.append(score[0])
    return word_tokenize_list, score_dict



In [193]:

def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/REATData/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))


def lstm_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()  
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))

    # attribution for the prediction
    weights = best_model.lstm.state_dict()
    W_ii, W_if, W_ig, W_io = np.split(weights['weight_ih_l0'].cpu().numpy(), 4, 0)
    W_hi, W_hf, W_hg, W_ho = np.split(weights['weight_hh_l0'].cpu().numpy(), 4, 0)
    b_i, b_f, b_g, b_o = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 4)


    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_if, x[i,0,:]) + np.matmul(W_hf, hn[i-1,0,:]) + b_f)
        z_dict.append(z)

    o_dict = []
    for i in range(length):
        if i == 0:
            o = sigmoid(np.matmul(W_io, x[i,0,:]) + b_o)
        else:
            o = sigmoid(np.matmul(W_io, x[i,0,:]) + np.matmul(W_ho, hn[i-1,0,:]) + b_o)
        o_dict.append(o)


    alpha_dict = []
    for i in range(len(z_dict)):
        if i == 0:
            alpha_dict.append(z_dict[0])
        else:
            alpha_dict.append(z_dict[i] * o_dict[i] / o_dict[i-1])

    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()
    target_class = pred_label


    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:]
        else:
            updating = hn[i,0,:] - alpha_dict[i] * hn[i-1,0,:]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]
        score = np.matmul( W[target_class], updating * forgetting) 
        score_dict.append(score[0])
    return word_tokenize_list, score_dict

In [194]:
# import torch
# import torch.nn as nn
from torch import optim
import time, random
import os
from tqdm import tqdm
# from lstm import LSTMSentiment
# from gru import GRUSentiment
# from bigru import BiGRUSentiment
from torchtext import data
import numpy as np
import argparse
# from torch.autograd import Variable
# import torch.nn.functional as F

torch.set_num_threads(8)
torch.manual_seed(1)
random.seed(1)


def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            if word in vocab:
               word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return word_vecs


def get_accuracy(truth, pred):
    assert len(truth) == len(pred)
    right = 0
    for i in range(len(truth)):
        if truth[i] == pred[i]:
            right += 1.0
    return right / len(truth)


def train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch):
    model.train()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    count = 0
    for batch in tqdm(train_iter, desc='Train epoch '+str(epoch+1)):
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()
        sent = sent.cuda()
        pred, _, _ = model(sent)
        pred = F.log_softmax(pred)
        pred = pred.cpu()
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        model.zero_grad()
        loss = loss_function(pred, label)
        # avg_loss += loss.data[0]
        avg_loss += loss.data
        count += 1
        loss.backward()
        optimizer.step()
    avg_loss /= len(train_iter)
    acc = get_accuracy(truth_res, pred_res)
    return avg_loss, acc


def evaluate(model, data, loss_function, name):
    model.eval()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    for batch in data:
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()
        sent = sent.cuda()
        pred, _, _ = model(sent)
        pred = F.log_softmax(pred)
        pred = pred.cpu()
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        loss = loss_function(pred, label)
        # avg_loss += loss.data[0]
        avg_loss += loss.data
    avg_loss /= len(data)
    acc = get_accuracy(truth_res, pred_res)
    print(name + ': loss %.2f acc %.1f' % (avg_loss, acc*100))
    return acc


def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/REATData/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1)
    ## for GPU run
#     train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
#                 batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=None)
    return train_iter, dev_iter, test_iter



# args = argparse.ArgumentParser()
# args.add_argument('--m', dest='model', default='lstm', help='specify the mode to use (default: lstm)')
# args = args.parse_args()

def runModel( modelName='lstm'):
    EPOCHS = 20
    USE_GPU = torch.cuda.is_available()
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 150

    BATCH_SIZE = 5
    timestamp = str(int(time.time()))
    best_dev_acc = 0.0


    # data is from torchtext.data, it is an object
    text_field = data.Field(lower=True)
    label_field = data.Field(sequential=False)
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, BATCH_SIZE)
    print(text_field, label_field)


    if modelName == 'lstm':
        print('model is lstm')
        model = LSTMSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                            use_gpu=USE_GPU, batch_size=BATCH_SIZE)

    if modelName == 'gru':
        print('model is gru')
        model = GRUSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                            use_gpu=USE_GPU, batch_size=BATCH_SIZE)

    if modelName == 'bigru':
        print('model is bigru')
        model = BiGRUSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                            use_gpu=USE_GPU, batch_size=BATCH_SIZE)

    if USE_GPU:
        print('use cuda')
        model = model.cuda()
        # .deatch().cpu().clone().numpy()


    print('Load word embeddings...')
    # word2vector
    word_to_idx = text_field.vocab.stoi
    # example of word_to_idx: u'schools': 14512, u'mimicking': 12930
    # len(word_to_idx) = 16190
    pretrained_embeddings = np.random.uniform(-0.25, 0.25, (len(text_field.vocab), 300))
    # pretrained_embeddings.shape = (16190, 300)

    pretrained_embeddings[0] = 0 #set all pretrained_embeddings[0] to 0
    word2vec = load_bin_vec('/content/gdrive/My Drive/twitter misinfo/news_vec/GoogleNews-vectors-negative300.bin', word_to_idx)
    for word, vector in word2vec.items():
        pretrained_embeddings[word_to_idx[word]-1] = vector
    # the embedding layer of this model is using the pretrained embeddings
    model.embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))
    # model.embeddings.weight.data = text_field.vocab.vectors
    # model.embeddings.embed.weight.requires_grad = False


    best_model = model
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_function = nn.NLLLoss()


    print('Training...')
    if modelName == 'gru':
        out_dir = "models/gru"
    elif modelName == 'lstm':
        out_dir = "models/lstm"
    else:
        out_dir = "models/bigru"

    #out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("Writing to {}\n".format(out_dir))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    for epoch in range(EPOCHS):
        avg_loss, acc = train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch)
        tqdm.write('Train: loss %.2f acc %.1f' % (avg_loss, acc*100))
        dev_acc = evaluate(model, dev_iter, loss_function, 'Dev')
        if dev_acc > best_dev_acc:
            if best_dev_acc > 0:
                os.system('rm '+ out_dir + '/best_model' + '.pkl')
            best_dev_acc = dev_acc
            best_model = model
            torch.save(best_model, out_dir + '/best_model' + '.pkl')
            # evaluate on test with the best dev performance model
            test_acc = evaluate(best_model, test_iter, loss_function, 'Test')


    return test_iter, loss_function

In [195]:
# !wget https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [196]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [197]:
# import zipfile


# zip_ref = zipfile.ZipFile("/content/GoogleNews-vectors-negative300.bin.gz", 'r')
# zip_ref.extractall("/content/gdrive/My Drive/twitter misinfo/REATData/")
# zip_ref.close()

# !tar -xvf  '/content/GoogleNews-vectors-negative300.bin.gz' -C '/content/gdrive/My Drive/twitter misinfo/REATData/'

In [198]:
# !unzip '/content/GoogleNews-vectors-negative300.bin.gz' -d '/content/gdrive/My Drive/twitter misinfo/REATData/'

In [199]:
# !gunzip '/content/GoogleNews-vectors-negative300.bin.gz'

In [200]:
!mv '/content/GoogleNews-vectors-negative300.bin' '/content/gdrive/My Drive/twitter misinfo/news_vec/'

mv: cannot stat '/content/GoogleNews-vectors-negative300.bin': No such file or directory


In [201]:
model_name = 'lstm'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


<torchtext.data.field.Field object at 0x7fe04113c588> <torchtext.data.field.Field object at 0x7fe04113cd68>
model is lstm
use cuda
Load word embeddings...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
Train epoch 1:   2%|▏         | 26/1384 [00:00<00:05, 259.77it/s]

Training...
Writing to models/lstm



Train epoch 1: 100%|██████████| 1384/1384 [00:04<00:00, 292.07it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LSTMSentiment. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train epoch 2:   0%|          | 0/1384 [00:00<?, ?it/s]

...

Train: loss 0.00 acc 100.0
Dev: loss 1.75 acc 76.6


Train epoch 20:   2%|▏         | 28/1384 [00:00<00:05, 270.30it/s]

Train: loss 0.00 acc 100.0
Dev: loss 1.95 acc 77.1


Train epoch 20: 100%|██████████| 1384/1384 [00:04<00:00, 283.08it/s]


Train: loss 0.00 acc 100.0
Dev: loss 2.13 acc 77.4
Final Test: loss 1.37 acc 78.2


In [202]:
model_name = 'gru'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


<torchtext.data.field.Field object at 0x7fe0729f1358> <torchtext.data.field.Field object at 0x7fe0729f1048>
model is gru
use cuda
Load word embeddings...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
Train epoch 1:   2%|▏         | 27/1384 [00:00<00:05, 262.87it/s]

Training...
Writing to models/gru



Train epoch 1: 100%|██████████| 1384/1384 [00:04<00:00, 291.87it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type GRUSentiment. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train epoch 2:   0%|          | 0/1384 [00:00<?, ?it/s]

...

Train: loss 0.00 acc 100.0
Dev: loss 1.93 acc 77.3


Train epoch 20:   2%|▏         | 27/1384 [00:00<00:05, 260.55it/s]

Train: loss 0.00 acc 100.0
Dev: loss 2.01 acc 77.4


Train epoch 20: 100%|██████████| 1384/1384 [00:04<00:00, 290.80it/s]


Train: loss 0.00 acc 100.0
Dev: loss 2.09 acc 77.5
Final Test: loss 0.45 acc 81.1


In [203]:
model_name = 'bigru'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


<torchtext.data.field.Field object at 0x7fe072202c50> <torchtext.data.field.Field object at 0x7fe139823c88>
model is bigru
use cuda
Load word embeddings...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
Train epoch 1:   1%|▏         | 20/1384 [00:00<00:07, 191.59it/s]

Training...
Writing to models/bigru



Train epoch 1: 100%|██████████| 1384/1384 [00:06<00:00, 209.23it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type BiGRUSentiment. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train epoch 2:   0%|          | 0/1384 [00:00<?, ?it/s]

...

Train: loss 0.00 acc 99.9
Dev: loss 2.12 acc 74.9


Train epoch 20:   2%|▏         | 21/1384 [00:00<00:06, 202.93it/s]

Train: loss 0.00 acc 100.0
Dev: loss 2.30 acc 75.2


Train epoch 20: 100%|██████████| 1384/1384 [00:06<00:00, 208.29it/s]


Train: loss 0.00 acc 100.0
Dev: loss 2.30 acc 74.5
Final Test: loss 0.43 acc 81.8


In [204]:
# testing text REAT GRU
text = "the fight scenes are fun but it grows tedious"
# text = "the story may be new, but it does not serve lots of laughs"
best_model = torch.load('models/gru/best_model.pkl')
word_tokenize_list, score_dict = gru_attribution(best_model, text) 


for i in range(len(score_dict)):
    print(word_tokenize_list[i], score_dict[i])

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


prediction category: positive sentiment with confidence of 0.5652975
the -0.0012669097229697055
fight 0.02889450546675601
scenes 0.012092458623083564
are -0.002933762114131487
fun 0.7640238165652878
but 0.03971543003468291
it 0.05716578755473795
grows -0.037601178074776996
tedious -0.6784781642950917


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [205]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [206]:
best_model

GRUSentiment(
  (embeddings): Embedding(16190, 300)
  (gru): GRU(300, 150)
  (hidden2label): Linear(in_features=150, out_features=2, bias=True)
)

In [207]:
# device = torch.device('cpu')

In [208]:
# testing text REAT LSTM
text = "the fight scenes are fun but it grows tedious"
# text = "the story may be new, but it does not serve lots of laughs"
best_model = torch.load('models/lstm/best_model.pkl')
word_tokenize_list, score_dict = lstm_attribution(best_model, text)


for i in range(len(score_dict)):
    print(word_tokenize_list[i], score_dict[i])

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


prediction category: positive sentiment with confidence of 0.9998524
the 0.00822999503301368
fight 1.6702979925432175
scenes -0.12405586189557385
are 0.058003913862864104
fun 1.4125126823478495
but 0.3106269395908062
it 0.20171123215519676
grows 0.8320851439000221
tedious 0.012343473319678064


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [209]:
# testing text REAT BiGRU
text = "the fight scenes are fun but it grows tedious"
#text = "the story may be new, but it does not serve lots of laughs"
best_model = torch.load('models/bigru/best_model.pkl')

word_tokenize_list, score_dict = bigru_attribution(best_model, text) 


for i in range(len(score_dict)):
    print(word_tokenize_list[i], score_dict[i])

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


prediction category: negative sentiment with confidence of 0.7518371
the 0.04165439800714155
fight -0.043117512648348524
scenes 0.005040971845929512
are -0.06339623710076298
fun -0.3209914180062183
but 0.06472793711242318
it -0.00028146558467827823
grows 0.34954928924486167
tedious 0.3983019658129142


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
